In [2]:
# Import Core Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Set Theme for Plotly Maps
# Different MapBox Styles
# carto-darkmatter
# open-street-map
# carto-positron

map_theme = 'carto-darkmatter'

# Set Theme for graphs
sns.set_theme(palette='bright')

# Set Pandas Parameters
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)

# Import DataFrame
stl_df = pd.read_csv('../Data/STL-Data-2023.csv')

# Create County DataFrame
stl_county_df = stl_df.loc[stl_df['Geo-County'] == 'Saint Louis County']

# Create City DataFrame
stl_city_df = stl_df.loc[stl_df['Geo-County'] == 'Saint-Louis City']

## Map-Cover Scripts

In [7]:
def area_MapPicture(data_frame, area_col, nibrs_col, image_location):

    # Create list of all NeighborHoods
    area_list = data_frame[data_frame[area_col].notnull()][area_col].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = data_frame[data_frame[area_col] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrs_col].value_counts().reset_index())

        area_vc_mappings = dict()

        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrs_col],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrs_col].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = area_col + ':' + '%{customdata[0]}'
        nibrs_hover = nibrs_col + ':' + '%{customdata[1]}'
                                                  
        # Create Geographic Map
        mapPlot = go.Figure(go.Scattermapbox(name=f'{area_col} Crime Points',                          # Chart Name
                             lat=area_latitudes,                                       # Latitudes
                             lon=area_longitudes,                                      # Longitudes
                             showlegend=False,             
                             marker=dict(color=area_df['NIBRS-Map']),
                             customdata=area_df[[area_col, nibrs_col]],                # Custom Data Columns for Hover
                             hovertemplate="<br>".join([area_hover, nibrs_hover])))     # Hover Information Template     

        mapPlot.update_layout(height= 900,                                                # Height of Subplot
                              width= 1400,                                                # Width of Subplot
                              margin={"r":0,"t":0,"l":0,"b":0},                        # Margin values right, top, left, bottom                  
                              autosize=True,                                              # Set Autosize of charts to True or False
                              template='plotly_dark',                                     # Template of Subplot
                              mapbox=dict(zoom=12.5,                                        # Zoom of Map
                                          pitch=0,                                        # Pitch of Map
                                          bearing=0,                                      # Bearing of Map
                                          style=map_theme,                                # Theme of Scatter Map
                                          center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                      lon=np.mean(area_longitudes))))     # Mean of Area Longitudes
        

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # #area_Subplot.show()
        mapPlot.write_image(f'{image_location}{area}.jpeg')

# Create Neighborhood HTML images
area_MapPicture(data_frame=stl_df,
                  area_col='Geo-Town',
                  nibrs_col='NIBRS-Offense',
                  image_location='Map-Covers/Towns/')

# Create Town HTML images
area_MapPicture(data_frame=stl_df,
                  area_col='Geo-Village',
                  nibrs_col='NIBRS-Offense',
                  image_location='Map-Covers/Villages/')

# Create Village HTML images
area_MapPicture(data_frame=stl_df,
                  area_col='Geo-Neighborhood',
                  nibrs_col='NIBRS-Offense',
                  image_location='Map-Covers/Neighborhoods/')

In [58]:
def stlWebLinks(area_col, dataFrame, folder_name):
    # Create pandas DF to store names and sort them into alphabetical order
    areaNames = pd.DataFrame(columns=['Name'],data=dataFrame[dataFrame[area_col].notnull()][area_col].unique()).sort_values('Name').reset_index(drop=True)
    areaNames['SpaceFixed'] = areaNames['Name'].str.replace(" ", "-")
    areaNames['TotalFixed'] = areaNames['SpaceFixed'].str.replace("/", "-")
    print(areaNames)

    for i in range(0,len(areaNames)):
        # Open file and set mode to append
        file1 = open("myfile.txt", "a")
    
        # writing newline character
        file1.write(f'''\n<!-- {areaNames['Name'][i]} -->
<a href="google.com">
    <div class="stl-port-box">
        <img src="Map-Covers/{folder_name}/{areaNames['TotalFixed'][i]}.jpeg" alt="{areaNames["Name"][i]} Crime Points">
        <div class="stl-port-layer">
            <h4>{areaNames['Name'][i]}</h4>
        </div>
    </div>
</a>''')
    # Close and save file
    file1.close()

stlWebLinks('Geo-Neighborhood', stl_df, 'Neighborhoods')

                Name        SpaceFixed        TotalFixed
0            Academy           Academy           Academy
1              Baden             Baden             Baden
2        Benton Park       Benton-Park       Benton-Park
3   Benton Park West  Benton-Park-West  Benton-Park-West
4          Bevo Mill         Bevo-Mill         Bevo-Mill
..               ...               ...               ...
73  Walnut Park East  Walnut-Park-East  Walnut-Park-East
74  Walnut Park West  Walnut-Park-West  Walnut-Park-West
75  Wells Goodfellow  Wells-Goodfellow  Wells-Goodfellow
76          West End          West-End          West-End
77    Wydown Skinker    Wydown-Skinker    Wydown-Skinker

[78 rows x 3 columns]
